# Login

In [5]:
username = 'fanja595'  # vitejej104@bzemail.com
password = 'QMN8PcCP'
code = 'ZBEA4H7DZMXAUSPRH35K453T4HMX4YVS'
sessionid = '62883843682%3A6jKzPXwBCyQpsv%3A7%3AAYew09eJpzcSsnXwP-yO3U75VB_NHMsBCodz7nn0pg1'

In [27]:
import pyotp

verification_code = pyotp.TOTP(code).now()
verification_code

'857233'

In [7]:
from instagrapi import Client

cl = Client()
cl.set_proxy('http://127.0.0.1:7890')
cl.login_by_sessionid(sessionid)
# cl.login(username, password, verification_code)

# user_id = cl.user_id_from_username(username)
# medias = cl.user_medias(user_id, 20)
# user_id

ClientConnectionError: ConnectionError HTTPSConnectionPool(host='i.instagram.com', port=443): Max retries exceeded with url: /api/v1/users/62883843682/info/ (Caused by ProtocolError('Connection aborted.', ConnectionAbortedError(10053, '你的主机中的软件中止了一个已建立的连接。', None, 10053, None)))

In [ ]:
cl.dump_settings('settings.json')
# cl.load_settings('settings.json')
# cl.get_timeline_feed()

# 获取粉丝

In [ ]:
user_id = '5011353332'  # trump
amount = 10

In [ ]:
def get_user_followers(user_id, amount):
    max_id = ''
    return_hits = 0
    while return_hits < amount:
        users, max_id = cl.user_followers_v1_chunk(user_id, max_amount=200, max_id=max_id)
        yield from users
        return_hits += len(users)
        if not max_id: 
            break

for user in get_user_followers(user_id, amount):
    userstr = f'{user.pk} {user.username} {user.full_name}\n'
    with open(f'{user_id}_{amount}.txt', 'a', encoding='utf-8') as fp:
        fp.write(userstr)

In [ ]:
cl.private_request(f"friendships/{user_id}/followers/",
                params={
                    "max_id": '',
                    "count": 12,
                    "rank_token": cl.rank_token,
                    "search_surface": "follow_list_page",
                    "query": "",
                    "enable_groups": "true",
                },)

In [ ]:
users, _ = cl.user_followers_v1_chunk(user_id, max_amount=10, max_id='')
for user in users:
    print(user.pk)

# 发送消息

In [ ]:
# cl.direct_send(text='Hello', user_ids=['62433374395'])
cl.direct_send('How are you?', user_ids=[cl.user_id])  # send youself

In [ ]:
import json

cl.private_request("https://i.instagram.com/api/v1/direct_v2/create_group_thread/",
                   data={
                    "_csrftoken": cl.token,
                    "_uuid": cl.uuid,
                    "_uid": cl.user_id,
                    "recipient_users": json.dumps(
                        [str(user) for user in ['4770627945', '63470766171', '49746075777']], separators=(",", ":")
                    ),
                },)

In [ ]:
from datetime import datetime

# 设定群组消息间隔时间
group_dm_delay = 30
# 查找最后建的群组, 确定最后发送消息时间
threads = cl.direct_threads()
group = None
last_active_at = None
meet_dm_delay = False  # 是否达到时间间隔要求
for thread in threads:
    if thread.is_group and thread.admin_user_ids[0] == cl.user_id:
        group = thread
        break
# 存在群组, 查找最近群组的最近20条群组消息, 获取管理员发布消息的最后时间
if group:
    print(group.id, len(group.users), group.thread_title)
    messages = cl.direct_messages(group.id)
    # 如果没有主动发送消息的最后时间, 则取最后一条群组消息的时间
    last_active_at = messages[0].timestamp
    for message in messages:
        if int(message.user_id) == cl.user_id:
            if message.item_type == 'media':
                print('[图片消息]', message.timestamp.isoformat(), message.media.thumbnail_url)
                last_active_at = message.timestamp
                break
            if message.item_type == 'xma_link':
                print('[文字消息]', message.timestamp.isoformat(), message.text)
                last_active_at = message.timestamp
                break
    print('最后管理员消息或群组活跃时间', last_active_at.isoformat())
    timedelta = (datetime.now() - group.messages[0].timestamp).total_seconds() // 60
    if timedelta >= group_dm_delay:
        meet_dm_delay = True
# 如果最后消息发送时间满足设定间隔(15分钟以上或更久), 则可以创建群组和发送消息
if not meet_dm_delay:
    print('不满足设定群组创建和消息时间, 请等待一段时间后重试')
else:
    print('TODO 创建新的群组')
    # cl.direct_send('Hello', thread_ids=[group.id])  # 发送文本消息
    # cl.direct_send_photo(photo_path, thread_ids=[group.id])  # 发送图片消息

In [ ]:
cl.direct_messages(group.id, 20)

In [ ]:
cl.direct_threads()[3]

In [ ]:
cl.direct_send('Happy happy', thread_ids=[cl.direct_threads()[3].id])

In [4]:

line = "sessionid=64055380614%3ApTBqTXlUuIq78K%3A22%3AAYfpIsHGBa_wO08jvcdJYdENF1UuWkUw3Vpe2jmAOw|"
import re
pattern = re.compile(r'(?<=sessionid=).*?(?=\|)')
if result := re.search(pattern, line):
    print( result.group(0) )


64055380614%3ApTBqTXlUuIq78K%3A22%3AAYfpIsHGBa_wO08jvcdJYdENF1UuWkUw3Vpe2jmAOw
